In [0]:
_git_branch = 'master'

#Document parser, refer https://github.com/nemoware/document-parser/releases
lib_version = '1.1.15'


# PROTCOLS analysre debug
1. IN the left-side panel, change to files tab 
2. Upload protocol PROTOCOL file
3. right click on uploaded file -> copy path
2. enter its path/name into the form below
3. runtime/run all

# INIT

## pull code

In [0]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from analyser.documents import TextMap
from analyser.legal_docs import DocumentJson
from colab_support.renderer import HtmlRenderer

 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:

import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from analyser.contract_parser import ContractAnlysingContext, ContractDocument
from analyser.contract_patterns import ContractPatternFactory
from analyser.legal_docs import LegalDocument
 
from analyser.ml_tools import *

# from headers_detector import doc_features, load_model, make_headline_attention_vector
from analyser.hyperparams import HyperParameters
from analyser.protocol_parser import protocol_votes_re
 

## 💅 Init Embedder(s)

In [0]:
# from protocol_parser import  ProtocolPatternFactory
from tf_support.embedder_elmo import ElmoEmbedder
# from contract_patterns import ContractPatternFactory
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")

# protocols_factory = ProtocolPatternFactory(elmo_embedder)
# contracts_factory = ContractPatternFactory(elmo_embedder)

In [0]:
from analyser.contract_parser import find_value_sign_currency_attention
from analyser.legal_docs import tokenize_doc_into_sentences_map, ContractValue
from analyser.ml_tools import *
from analyser.parsing import ParsingContext
from analyser.patterns import *
from analyser.protocol_parser import ProtocolDocument, find_confident_spans, protocol_votes_re, ProtocolPatternFactory
from analyser.protocol_parser import  find_org_structural_level, find_protocol_org, ProtocolParser
from analyser.text_tools import *

# legal_docs.py
from tf_support.embedder_elmo import ElmoEmbedder
 



In [0]:
protocol_analyser = ProtocolParser(elmo_embedder, elmo_embedder_default)

# Read doc

In [0]:
#@title ✅Pre-embedded or Upload?

filename = '/content/3.\u043F\u0440\u043E\u0442\u043E\u043A\u043E\u043B.doc' #@param {type:"string"}


results = wp.read_doc(filename)
for doc in results['documents'][:1]:  # XXX
  if 'PROTOCOL' == doc['documentType']:    
    doc = join_paragraphs(doc, 'no_id')


for p in doc.paragraphs:
  print ('☢️', p.header.value.strip())

### Embedd and back up pickle (TODO: fix 💀)

In [0]:

# protocol_analyser.ebmedd(doc)        
# with open(f'{filename}.pickle', 'wb') as handle:
#   pickle.dump(doc, handle, protocol=pickle.HIGHEST_PROTOCOL)
  

# Analyse PHASE 1

In [0]:
# =====================================
from analyser.parsing import AuditContext
actx = AuditContext()
protocol_analyser.find_org_date_number(doc, actx)

### render PHASE 0 results

In [0]:
for t in doc.get_tags():
  print(t)
renderer_.render_color_text(doc.tokens,  doc.get_tags_attention() )



#PHASE 1

In [0]:
protocol_analyser.find_attributes(doc, actx)

### render PHASE 1 results

In [0]:
for t in doc.get_tags():
  print(t)
renderer_.render_color_text(doc.tokens,  doc.get_tags_attention() )

In [0]:
import re
from analyser.dates import document_number_c, document_number_valid_c
def find_document_number_in_subdoc(doc: LegalDocument, tagname='number', parent=None) -> [SemanticTag]:

  ret =[]
  findings = re.finditer(document_number_c, doc.text)
  if findings:
    for finding in findings:
      print(finding)
      _number = finding['number']
      if document_number_valid_c.match(_number):
        span = doc.tokens_map.token_indices_by_char_range(finding.span())
        tag =  SemanticTag(tagname, _number, span, parent=parent)
        tag.offset(doc.start)
        ret.append(tag)
      else:
        print('invalid', _number)
  
  return ret

def find_contract_numbers(self, doc) -> [ContractValue]:

  values = []
  for agenda_question_tag in doc.agenda_questions:
    subdoc = doc[agenda_question_tag.as_slice()]
    # print(subdoc.text)
    numbers = find_document_number_in_subdoc(subdoc, tagname='number', parent=agenda_question_tag)

    for k, v in enumerate(numbers):
      # v.parent = agenda_question_tag
      v.kind = SemanticTag.number_key(v.kind, k)
    values += numbers
  return values
 

find_contract_numbers(protocol_analyser, doc)

In [0]:
doc.to_json_obj()['attributes']

# Debug

## Sections attention

In [0]:


protocol_sections_edges = protocol_analyser.find_protocol_sections_edges(doc.distances_per_sentence_pattern_dict)
renderer_.render_color_text(doc.sentence_map.tokens,  protocol_sections_edges , _range=[0,1], separator='¶<br>')


### sections spans

In [0]:
from analyser.documents import sentences_attention_to_words
from analyser.dates import   document_number_c
from analyser.contract_agents import complete_re as agents_re
from analyser.transaction_values import complete_re as values_re


### Print all sections

In [0]:
# tags = protocol_analyser.find_question_decision_sections(doc)
# for t in tags:
#   print('='*100)
#   print(t.get_key().upper()+'>>>>>>>>>>>>')
#   print(doc.substr(t), t.get_key())

In [0]:
### votes and numbers

In [0]:

#DEAL APPROVAL SENTENCES
v_deal_approval = max_exclusive_pattern_by_prefix(doc.distances_per_sentence_pattern_dict, 'deal_approval_')
_spans, deal_approval_av = sentences_attention_to_words(v_deal_approval, doc.sentence_map,
                                                                        doc.tokens_map)
deal_approval_relu_av = best_above(deal_approval_av, 0.5)

# VOTES
votes_av = doc.tokens_map.regex_attention(protocol_votes_re)
# DOC NUMBERS
numbers_av = doc.tokens_map.regex_attention(document_number_c)
# DOC AGENTS orgs
agents_av = doc.tokens_map.regex_attention(agents_re)

# DOC MARGIN VALUES
margin_values_av = protocol_analyser._get_value_attention_vector(doc)
margin_values_v = doc.tokens_map.regex_attention(values_re)
margin_values_v*=margin_values_av


renderer_.render_color_text(doc.tokens,  deal_approval_relu_av , _range=[0,1])

## Combined attention

In [0]:
combined_av = sum_probabilities([deal_approval_relu_av,
                                 margin_values_v, 
                                 agents_av/2,
                                 votes_av/2, 
                                 numbers_av/2])


combined_av_norm = combined_av = best_above(combined_av, 0.2) 
renderer_.render_color_text(doc.tokens,  combined_av_norm , _range=[0,1])

In [0]:
# v_sections_attention = find_protocol_sections_edges(protocol_analyser, doc.distances_per_sentence_pattern_dict)

_question_spans_sent = spans_between_non_zero_attention(protocol_sections_edges)
question_spans_words = doc.sentence_map.remap_slices(_question_spans_sent, doc.tokens_map)
agenda_questions = list(find_confident_spans(question_spans_words, combined_av_norm, 'agenda_item', 0.5))

for x in agenda_questions:
  print("="*100)
  print(x)
  print(doc.substr(x))

for span in question_spans_words:
  print("="*100)
  sl=slice(span[0],span[1])
  renderer_.render_color_text(doc[sl].tokens,  combined_av_norm[sl] , _range=[0,1])

In [0]:
raise ('stop here please for now')

In [0]:

fig = plt.figure()
ax = plt.axes()

ax.plot(deal_approval_relu_av,  color=(1.0,0.2,0.3))   
ax.plot(best_above(deal_approval_relu_av, 0.4),  color=(0.0,1.0,0.3))   

## Debug votes finder

In [0]:
from analyser.protocol_parser import ProtocolAV
renderer_.render_color_text(doc.tokens,  doc.distances_per_pattern_dict[ProtocolAV.bin_votes_attention.name] , _range=[0,1])


In [0]:

renderer_.render_color_text(doc.tokens, numbers_av+votes_av, _range=[0,2])


### debug protocol_votes_re

In [0]:
x = protocol_votes_re.search(doc.text)

match = doc.text[x.span()[0]:x.span()[1]]
print(f'[{match}]')

### debug spans_having_votes_words

In [0]:
v_sections_attention = protocol_analyser.find_protocol_sections_edges(doc.distances_per_sentence_pattern_dict)
question_spans_sent = spans_between_non_zero_attention(v_sections_attention)
question_spans_words = doc.sentence_map.remap_slices(question_spans_sent, doc.tokens_map)

for c in question_spans_words:
  print('-'*80)
  print (c, doc.tokens_map.text_range(c))

In [0]:



_spans_having_votes = list(protocol_analyser.collect_spans_having_votes(question_spans_sent, doc.sentence_map))
spans_having_votes_words = doc.sentence_map.remap_slices(_spans_having_votes, doc.tokens_map)
x=spans_having_votes_words[0]
# x

# print(protocol_votes_re)
print(doc.tokens[x[0]:x[1]])

In [0]:
protocol_analyser.collect_spans_having_votes

In [0]:
tags = protocol_analyser.find_question_decision_sections(doc)
wa=doc.distances_per_pattern_dict
# doc.distances_per_pattern_dict['relu_deal_approval']
# bc=wa['relu_deal_approval'], wa['bin_votes_attention'] / 5]
block_confidence = sum_probabilities([wa['numbers_attention'], wa['relu_deal_approval'], wa['bin_votes_attention'] / 5])
renderer_.render_color_text(doc.tokens,  wa['bin_votes_attention'], _range=(0,1))

In [0]:
# class ProtocolAttentionVectors(Enum):
#   numbers_attention=1,

In [0]:

#helpers


# def _paint_tags(self, tags) -> [SemanticTag]:
   
#   _attention = np.zeros(self.__len__())

#   for t in tags:
#     _attention[t.as_slice()] += t.confidence
#   return _attention
 
 

In [0]:
# protocol_analyser.find_values(doc)

for mv in doc.margin_values:
  print(mv.parent)

In [0]:
# def _get_value_attention_vector(self, doc: LegalDocument):
#   s_value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_p_')
  
#   doc.distances_per_pattern_dict [ '__max_value_av' ] = s_value_attention_vector #just for debugging
#   not_value_av = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'not_sum_')
  
#   not_value_av = smooth_safe(not_value_av, window_len=5)
  
#   not_value_av = relu(not_value_av, 0.5)
#   doc.distances_per_pattern_dict['__not_value_av'] = not_value_av #just for debugging


#   s_value_attention_vector -= not_value_av * 0.8
#   s_value_attention_vector = relu(s_value_attention_vector, 0.3)
#   return s_value_attention_vector

# v= _get_value_attention_vector(None, doc)


wa = doc.distances_per_pattern_dict  # words attention
v_sections_attention = protocol_analyser.find_protocol_sections_edges(doc.distances_per_sentence_pattern_dict)

# --------------
question_spans_sent = spans_between_non_zero_attention(v_sections_attention)
question_spans_words = doc.sentence_map.remap_slices(question_spans_sent, doc.tokens_map)
# --------------

# *More* attention to spans having votes
spans_having_votes = list(protocol_analyser.collect_spans_having_votes(question_spans_sent, doc.sentence_map))

spans_having_votes_words = doc.sentence_map.remap_slices(spans_having_votes, doc.tokens_map)
# questions_attention =  spans_to_attention(question_spans_words, len(doc))
wa['bin_votes_attention'] = spans_to_attention(spans_having_votes_words, len(doc))

# v_deal_approval_words = sentence_map.remap_spans(v_deal_approval,  doc.tokens_map )
v_deal_approval = max_exclusive_pattern_by_prefix(doc.distances_per_sentence_pattern_dict, 'deal_approval_')
_spans, v_deal_approval_words_attention = sentences_attention_to_words(v_deal_approval, doc.sentence_map,
                                                                        doc.tokens_map)

## value attention

wa['value_relu'] = protocol_analyser._get_value_attention_vector(doc)
wa['deal_approval_relu05'] = relu(v_deal_approval_words_attention, 0.5)


wa['value_in_context'] = sum_probabilities(
      [ wa['value_relu'],
       smooth_safe(wa['deal_approval_relu05']),
       smooth_safe(wa['bin_votes_attention']) / 3.0])

# wa [ 'value_in_context' ]
renderer_.render_color_text(doc.tokens,  wa [ '__not_value_av' ], _range=(0,1))

In [0]:
# def _get_value_attention_vector(self, doc: LegalDocument):
#   # vectors = list(filter_values_by_key_prefix(doc.distances_per_pattern_dict, 'sum_max_p_'))
  
#   s_value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_p_')
#   doc.distances_per_pattern_dict [ '__max_value_av' ] = s_value_attention_vector #just for debugging
#   s_value_attention_vector_neg = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_neg')
  
#   s_value_attention_vector_neg = relu(s_value_attention_vector_neg, 0.33)
#   doc.distances_per_pattern_dict['__not_value_av'] = s_value_attention_vector_neg #just for debugging
#   s_value_attention_vector -= s_value_attention_vector_neg * 0.66
#   s_value_attention_vector = relu(s_value_attention_vector, 0.25)
#   return s_value_attention_vector

# v = _get_value_attention_vector(protocol_analyser, doc)

# # print(doc.distances_per_pattern_dict.['relu_value_attention_vector'])

# # vectors = list(filter_values_by_key_prefix(doc.distances_per_pattern_dict, 'sum_max_neg'))
# # s_value_attention_vector =sum_probabilities (vectors)

f = protocol_analyser.protocols_factory
f.create_pattern('not_sum_0', ('', 'пункт 0.', ''))
f.create_pattern('not_sum_ad.1', ('', '190000 , Санкт-Петербург', ''))
f.create_pattern('not_sum_ad.2', ('', 'дом 0', ''))
f.create_pattern('not_sum_ad.3', ('', 'улица 0', ''))
f.create_pattern('not_sum_1', ('', '0 дней', ''))
f.create_pattern('not_sum_1.1', ('', 'в течение 0 ( ноля ) дней', ''))
f.create_pattern('not_sum_1.2', ('', '0 января', ''))
f.create_pattern('not_sum_2', ('', '0 минут', ''))
f.create_pattern('not_sum_3', ('', '0 часов', ''))
f.create_pattern('not_sum_4', ('', '0 процентов', ''))
f.create_pattern('not_sum_5', ('', '0 %', ''))
f.create_pattern('not_sum_5.1', ('', '0 % голосов', ''))
f.create_pattern('not_sum_6', ('', '2000 год', ''))
f.create_pattern('not_sum_7', ('', '0 человек', ''))
f.create_pattern('not_sum_8', ('', '0 метров', ''))


protocol_analyser.protocols_factory.embedd()

doc.calculate_distances_per_pattern(protocol_analyser.protocols_factory)
not_value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'not_sum_')
not_value_attention_vector = relu(not_value_attention_vector, 0.5)
renderer_.render_color_text(doc.tokens,  not_value_attention_vector, _range=(0,1))

In [0]:
 
values: [ContractValue] = find_value_sign_currency_attention(doc, doc.distances_per_pattern_dict ['relu_value_attention_vector'])
renderer_.render_color_text(doc.tokens,  doc.distances_per_pattern_dict ['relu_value_attention_vector'] )

### Json

In [0]:
jj = doc.to_json()
parsed = json.loads(jj)
del parsed['tokenization_maps']
print(json.dumps(parsed, indent=4, sort_keys=True))

### 👨🏿‍🚀Try find subject

In [0]:
subject1 = '''одобрять договора пожертвования денежных средств'''
subject2 = '''сумма 0 (миллионов) рублей 0 копеек '''
  
subject_embeddings = elmo_embedder_default.embedd_strings([subject1, subject2 ])

# import scipy.spatial.distance as distance



contract_subj_attention1 = calc_distances_to_pattern(doc.sentences_embeddings , subject_embeddings[0])
contract_subj_attention2 = calc_distances_to_pattern(doc.sentences_embeddings , subject_embeddings[1])
contract_subj_attention =  contract_subj_attention1# relu(contract_subj_attention1, 0.5)



renderer_.render_color_text(doc.sentence_map.tokens, contract_subj_attention, _range=(0,1), separator='<br>')



## Split text by agenda questions



### 📝 Sections attention

In [0]:
def find_protocol_sections_edges(distances_per_pattern_dict):

  patterns = ['deal_approval_', 'footers_', 'question_']
  vv_=[]
  for p in patterns:
    v_ = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, p)
    v_ = relu(v_, 0.5)
    vv_.append(v_)
  
  v_sections_attention = sum_probabilities(vv_)

  # v_deal_approval = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'deal_approval_')
  # v_deal_approval = relu(v_deal_approval, 0.5)

  # v_question = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'question_')
  # v_question = relu(v_question, 0.5)

  # v_sections_attention = sum_probabilities([v_deal_approval, v_question])
  v_sections_attention = relu(v_sections_attention, 0.7)
  return v_sections_attention


distances_per_pattern_dict = calc_distances_per_pattern_dict(sentences_embeddings_, patterns_dict, patterns_embeddings_)
v_sections_attention = find_protocol_sections_edges(distances_per_pattern_dict)

# renderer_.render_color_text(sentence_map.tokens , v_question+v_deal_approval, _range=(0,1), separator='<br>')
renderer_.render_color_text(sentence_map.tokens,  v_sections_attention, _range=(0,1), separator='<br>')
 

In [0]:
question_spans_sent = spans_between_non_zero_attention(v_sections_attention)
print('question_spans_sent', question_spans_sent)

question_spans_words =  sentence_map.remap_slices(question_spans_sent,  doc.tokens_map )
print('question_spans_words', question_spans_words)

More attention to spans having votes

In [0]:
spans_having_votes = list(collect_spans_having_votes(question_spans_sent, sentence_map))


spans_having_votes_words =  sentence_map.remap_slices(spans_having_votes,  doc.tokens_map )
# questions_attention =  spans_to_attention(question_spans_words, len(doc))
votes_attention =  spans_to_attention(spans_having_votes_words, len(doc))

# v_deal_approval_words = sentence_map.remap_spans(v_deal_approval,  doc.tokens_map )
v_deal_approval = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'deal_approval_')
_spans, v_deal_approval_words_attention =  sentences_attention_to_words(v_deal_approval, sentence_map,  doc.tokens_map)




### debug -------------votes_attention

renderer_.render_color_text(doc.tokens_map.tokens , relu(v_deal_approval_words_attention, 0.3)   , _range=(0,1))

for span in spans_having_votes:
  print(span, '=' * 50)
  subdoc = sentence_map.slice(span)
  print(subdoc.text)
  print(sentence_map.char_range( [span.start, span.stop] ))
  print('^' * 50)
  


### Value attention

In [0]:

s_value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_p_')
s_value_attention_vector_neg = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_neg')
s_value_attention_vector -=   s_value_attention_vector_neg/3
s_value_attention_vector = relu(s_value_attention_vector, 0.25)

v_deal_approval_words_attention = relu (v_deal_approval_words_attention, 0.5)
value_attention_vector = sum_probabilities ( [s_value_attention_vector, v_deal_approval_words_attention, votes_attention/3.0] )
# value_attention_vector = sum_probabilities ( [ v_deal_approval_words_attention, votes_attention/10.0] )
value_attention_vector = relu(value_attention_vector, 0.5)
 
renderer_.render_color_text(doc.tokens_map.tokens, value_attention_vector  )

In [0]:
#sentences to words
from contract_parser import find_value_sign_currency_attention 
from legal_docs import ContractValue

words_spans_having_votes =  sentence_map.remap_slices (spans_having_votes,  doc.tokens_map )


#--------------
#color things
print('spans_having_votes', list(spans_having_votes))
print('words_spans_having_votes', list(words_spans_having_votes))

#

# for span in list(words_spans_having_votes):
#   print(span)
#   selection [ span[0]:span[1]+1  ] += 0.4
#--------------

#find contract values
# for span in words_spans_having_votes:
#   subdoc = doc.subdoc_slice(slice(span[0],span[1]) )
values:List[ContractValue] = find_value_sign_currency_attention(doc, value_attention_vector)

#color 
numbers_attention = np.zeros(len(doc.tokens_map))
numbers_confidence = np.zeros(len(doc.tokens_map))
for v in values:
  numbers_confidence [ v.value.as_slice() ] +=v.value.confidence
  numbers_attention  [ v.value.as_slice() ] = 1
  numbers_attention  [ v.currency.as_slice() ] = 1
  numbers_attention  [ v.sign.as_slice() ] = 1

block_confidence = sum_probabilities ( [numbers_attention, v_deal_approval_words_attention, votes_attention/5] )

# value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, '_phrase')
renderer_.render_color_text(doc.tokens_map.tokens, numbers_attention, _range=(0,1))


## Collect only meaningful sections

In [0]:
# block_confidence = sum_probabilities ( [ numbers_attention, value_attention_vector, v_deal_approval_words_attention] )

renderer_.render_color_text(doc.tokens_map.tokens, block_confidence, _range=(0,1))


In [0]:
def find_confident_spans(slices, block_confidence, tag_name):
  k = 0
  for _slice in slices:
    k+=1
    pv = block_confidence[_slice[0]:_slice[1]]
    confidence = estimate_confidence_by_mean_top_non_zeros(pv, 5)
    print('-' * 100)
    print(_slice, confidence)
    if confidence > 0.6:
      st = SemanticTag(f"{tag_name}_{k}", None, _slice)
      st.confidence = confidence
      yield (st)


tags = find_confident_spans(question_spans_words, block_confidence, 'agenda_item' )
for tag in tags:
  print(tag.kind)
  print(doc.substr(tag))

# MISCL

In [0]:
def detect_agenda(distances_per_pattern_dict):
  v1 = max_exclusive_pattern_by_prefix(distances_per_pattern_dict,  'agenda_start_' )
  agenda_index_start = np.argmax(v1)
  # v =max_exclusive_pattern_by_prefix( [distances_per_pattern_dict['deal_approval_1'], distances_per_pattern_dict['deal_approval_2']]  )
  v2 = max_exclusive_pattern_by_prefix(distances_per_pattern_dict,  'agenda_end_' )
  agenda_index_end = np.argmax(v2)

  v2[:agenda_index_start] = 0
  v2[agenda_index_end:] = 0


  return None, v2
  pass

questions, attention_v = detect_agenda(distances_per_pattern_dict)
print(questions)
renderer_.render_color_text(spans_map.tokens , attention_v, _range=(0,1), separator='<br>')
# plot_similarity(messages_, message_embeddings_, 90)

In [0]:
doc[0:4].text

In [0]:

from contract_agents import find_org_names
from typing import List

def find_protocol_org(protocol)->List[SemanticTag]:
  ret=[]
  x:List[SemanticTag] = find_org_names(protocol[0:200])
  ret.append( SemanticTag.find_by_kind(x, 'org.1.name'))
  ret.append( SemanticTag.find_by_kind(x, 'org.1.type'))
  return x

x=find_protocol_org(doc)
for c in x:
  print(c)

## topic_attention

In [0]:
topic_attention = make_topic_attention(doc, headline_attention_vector, 'p_agenda_')
topic_attention = relu(topic_attention, 0.5)
# renderer_.render_color_text(doc.tokens_cc, topic_attention)

def mark_section(topic_attention:FixedVector) -> FixedVector:
  section_paint = np.zeros_like(topic_attention)
  start = np.argmax(topic_attention)
  start_v=topic_attention[start]
  section_paint[start:] = start_v

  return section_paint


  print(start)

_sp = mark_section(topic_attention)
renderer_.render_color_text(doc.tokens_cc, _sp, _range=(0,1))

In [0]:

all_headlines_indices = all_headlines_spans[:,0]

# ---
def find_sections_spans_by_indices_2(_doc: LegalDocument,
                                   headline_pattern_names,
                                   all_headlines_indices,
                                   headlines_mask,
                                   topic,
                                   relu_threshold=0.6):
  av_name = f'{topic}_headlines_attention_vector'
  print("TOPIC:", av_name)
  pv = _doc.distances_per_pattern_dict
  pv[av_name] = relu(combined_attention_vectors(pv, headline_pattern_names), relu_threshold)

  # masking
  pv[av_name] *= headlines_mask

  _headlines_of_interest_spans = find_sentences_with_attention(_doc.tokens_map, pv[av_name])
  _headlines_of_interest_indices = _headlines_of_interest_spans[:, 0]

  for i in _headlines_of_interest_indices:
    last = find_first_gt(i, all_headlines_indices)
    if last is None:
      last = len(_doc.tokens_map) - 1
    yield [i, last]
  pass


##==========
solution_spans = find_sections_spans_by_indices_2 (
    doc, solution_pattern_names, 
    all_headlines_indices,
    headline_attention_vector_relu, 
    topic='solutions', 
    relu_threshold=0.6)

agenda_spans   = find_sections_spans_by_indices_2 (
    doc, agenda_pattern_names, 
    all_headlines_indices, 
    headline_attention_vector_relu, 
    topic='agenda', 
    relu_threshold=0.6)

_agenda_hd_v = doc.distances_per_pattern_dict['agenda_headlines_attention_vector']
_solutions_hd_v = doc.distances_per_pattern_dict['solutions_headlines_attention_vector']

for agenda_span in agenda_spans:
  print('agenda_span', agenda_span)
  # print(doc.tokens_map.text_range(agenda_span))
  _slice = slice(agenda_span[0], agenda_span[1])
  renderer_.render_color_text(doc.tokens_map [_slice ], _filtered_headline_attention_vector[_slice])

print('*'*100)

print('='*100)
for solution_span in solution_spans:
  print(':::::: solution_span', solution_span)
  print('- '*50)
  # print('☢️ ', doc.tokens_map.text_range(solution_span))
  _slice = slice(solution_span[0],solution_span[1])
  renderer_.render_color_text(doc.tokens_map [_slice ], _filtered_headline_attention_vector[_slice])
  print('-'*100)

 

In [0]:
list(doc.distances_per_pattern_dict.keys())

In [0]:
renderer_.render_color_text(doc.tokens_cc, _solutions_hd_v)

# Test on uploaded file

In [0]:
# del doc

In [0]:
from integration.word_document_parser import WordDocParser, join_paragraphs

document_parser_v='1.0.9'

!wget https://github.com/nemoware/document-parser/releases/download/$document_parser_v/document-parser-$document_parser_v-distribution.zip
!unzip document-parser-$document_parser_v-distribution.zip  
os.environ ['documentparser']=f'/content/document-parser-{document_parser_v}'

wordDocParser = WordDocParser()

## read file

In [0]:
json_res = wordDocParser.read_doc('/content/Протокол_ООО Вектор продажа недвиж.doc')


In [0]:
from headers_detector import make_headline_attention_vector
from integration.word_document_parser import join_paragraphs
from ml_tools import find_non_zero_spans, merge_colliding_spans, relu


def analyse_protocol(json_res, ctx):
  #embedding, etc
  protocol = join_paragraphs(json_res, '0')
  protocol.embedd_tokens(ctx.protocols_factory.embedder)
  #TODO: tetect agents
  vd = protocol.calculate_distances_per_pattern(ctx.protocols_factory, verbosity=0)
  
  vd['headline_attention_vector'] = make_headline_attention_vector(protocol)
  vd['headline_attention_vector_relu'] = relu(vd['headline_attention_vector'], HyperParameters.parser_headline_attention_vector_denominator)

  all_headlines_spans = find_sentences_with_attention(protocol.tokens_map, vd['headline_attention_vector_relu'])
  
  filtered_all_headlines_spans = merge_colliding_spans(all_headlines_spans)
  all_headlines_indices = filtered_all_headlines_spans[:,0]

  # test
  vd['filtered_headline_attention_vector'] = np.zeros_like(vd['headline_attention_vector_relu'])
  for s in filtered_all_headlines_spans:
    vd['filtered_headline_attention_vector'][s[0]:s[1]] += 1

  return protocol, all_headlines_indices


protocol, all_headlines_indices = analyse_protocol(json_res, ctx)




## Solutions (решения по вопросам)

In [0]:

solution_spans = find_sections_spans_by_indices (protocol, 
                                                 solution_pattern_names, 
                                                 all_headlines_indices,
                                                 protocol.distances_per_pattern_dict['headline_attention_vector_relu'], 
                                                 'solution',
                                                  relu_threshold=0.75)


print('='*100)
for solution_span in solution_spans:
  print(':::::: solution_span', solution_span)
  print('- '*50)
  print('☢️ ', protocol.tokens_map.text_range(solution_span))
  print('-'*100)



## headline_attention_vector 


In [0]:
renderer_.render_color_text(protocol.tokens_cc, protocol.distances_per_pattern_dict['headline_attention_vector'])

## headline_attention_vector_relu

In [0]:
renderer_.render_color_text(protocol.tokens_cc, protocol.distances_per_pattern_dict['headline_attention_vector_relu'])

## filtered_headline_attention_vector

# Latent mean (center) sense

In [0]:
TextDocument = LegalDocument #alias

In [0]:
from patterns import *

def latent_sense(embeddings, wnd_half):
   
  R = np.zeros(len(embeddings))
  for i in range (len(embeddings)):
    _start = 0
    if i > wnd_half: _start = i-wnd_half
    word_emb = embeddings[i:i+1]
    window_emb = embeddings[_start : i+wnd_half]   
    semantic_similarity =  1- dist_mean_cosine(window_emb, word_emb)
    R[i] = semantic_similarity
  
  return R

def render_any(t, v):
  renderer_.render_color_text(t, v, _range=(min(v), max(v)) )


In [0]:
 
 
td = doc
td.parse()
td.embedd_tokens(elmo_embedder)
td=doc


local_sense = latent_sense(td.embeddings,  7 ) 
# mean_sense -= min(mean_sense)


# R
# mean_sense = relu(R, np.mean(R)*1.1)

render_any(td.tokens_cc,  local_sense )
render_any(td.tokens_cc,  local_sense-mean_sense)


# meaninful_tokens = np.nonzero(R)[0:][0]
# print('-'*100)
# print(f'tokens: {len(td.tokens)} \t meaninful {len(meaninful_tokens)}')
# print('-'*100)
# res=[ td.tokens_map.text_range([i, i+1]) for i in meaninful_tokens ]
# print(' ' .join(res) )

### Latent mean (mean-ing)

In [0]:
mean_sense = latent_sense(td.embeddings,  120 )

meaningless = relu (mean_sense, (np.max(mean_sense) + np.mean(mean_sense))/2  )
outliers = relu (1.0 - mean_sense,0.8)

render_any(td.tokens_cc, meaningless)

In [0]:
# 


### Latent local mean (mean-ing) without nonsense

In [0]:
mean_local_sense = latent_sense(td.embeddings,  7 )
nonsense_subtract = mean_local_sense - meaningless
nonsense_subtract = relu(nonsense_subtract, 0.2)
render_any(td.tokens_cc, nonsense_subtract )

# Pattern search (distance functions)

In [0]:
test_pattern = ctx.protocols_factory.create_pattern('test', ('','Принятие решения о заключении ООО « Технологический центр « Бажен » договора аренды недвижимого имущества с Обществом с ограниченной ответственностью « Ренова Лаб » ;',''))
test_pattern2 = ctx.protocols_factory.create_pattern('test2', ('','По вопросу № 7',''))
ctx.protocols_factory.embedd()

In [0]:
# def per_token_dist_mean_cosine(window, pattern):
   
#   d=0
  
#   for p in pattern:
#     for v in window:
#       d += distance.cosine(p, v)
  
#   return d / (len(window) * len(pattern))
def dist_cosine_to_point( text_emb, pt ):
  t_distances = np.zeros(len(text_emb))
  for i in range(len(text_emb)):
    # print(text_emb[i][0].shape, pt[0].shape)
    d = distance.cosine (text_emb[i], pt) 
    if d>1: d=1    
    t_distances[i] = d

  # print(min(t_distances), max(t_distances))
  
  return relu(t_distances,0.5)

def per_token_similarity_cosine( text_emb, pattern_emb ):   
  a_distances = np.zeros(len(text_emb))
  for p in pattern_emb:
    t_distances = 1 - dist_cosine_to_point(text_emb, p)    
    # a_distances = sum_probabilities ([ t_distances, a_distances ])
    a_distances = max_exclusive_pattern ([ t_distances, a_distances ])
    
  return a_distances

test_dists = per_token_similarity_cosine(doc.embeddings, test_pattern.embeddings )
# test_dists = relu (1-test_pattern._eval_distances(doc.embeddings), 0.5)
test_dists = relu(test_dists, 0.5)
render_any(td.tokens_cc, test_dists)



# distance.cosine (test_pattern.embeddings[1], doc.embeddings[1]) 

# C CHARTER

In [0]:
results = wp.read_doc('/content/МНГ Устав.docx')
for d in results['documents'][:1]:  # XXX
  if 'CHARTER' == d['documentType']:    
    charter = join_paragraphs(d, 'no_id')


# from legal_docs import tokenize_doc_into_sentences_map
sentence_map = tokenize_doc_into_sentences_map(charter, 250)

for token in sentence_map.tokens[0:30]:
  print('-->', token)

In [0]:
def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for hd in cd.headers:    
    span = hd['span']      
    markup_vector[span[0]:span[1]] += 2

  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector)

 



json_obj:DocumentJson = DocumentJson.from_json( charter.to_json())

json_obj.attributes['d']  = SemanticTag('kind', 'value', (7982, 8951))
html = json2html(json_obj)
display(HTML(html))



# 7982

v = np.zeros(len(charter))
# test_dists = relu (1-test_pattern._eval_distances(doc.embeddings), 0.5)
# v = relu(test_dists, 0.5)
# renderer_.render_color_text(charter.tokens, v)
# show_json(charter)

In [0]:
json_obj.attributes